In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load the Dataset
Then have a look at its structure..

You will notice that this data is clean, without any missing values, NaNs, nulls, etc.
Every column is properly named (including also the unit used to meassure each variable).
All values are floats.

In [ ]:
df = pd.read_csv("/kaggle/input/turbofan-hpc-efficiency/Turbofan_HPC_Efficiency.csv")
df.head(5)

# Let's dig a bit deeper

In [ ]:
# Some quick description of our data
df.describe()

In [ ]:
# Lets look how linearly correlated our features are
from seaborn import heatmap
heatmap(df.corr(), xticklabels=df.columns, yticklabels=df.columns)

***It was to be expected..***

Our data is highly linearly correlated. Almost every feature of this data set is perfectly correlated with each other (except from the HP Shaft RMPs, which remain constant).

Our target value, on the other hand, is inversly correlated with these features. 

# A small side note..

**Are you familiar with Aeroengine maintenance and Aeroplanes overhauls?**
Maintenance is mostly corrective, not predictive. Some predictives actions are taken (such as on-wing engine washing), but maintenance mostly relies on visual inspection and expert's knowledge.

**But why don't they just meassure efficiency?**
That would be a fair question, sadly that is not possible, it is not even possible to meassure real thrust (at least not easily). Furhtermore, not all of the variables present on this dataset are usually meassure, only some of them.

Usually, during a plane overhaul or engine maintenance these variables are meassured:
**Shaft speeds, Fuel Flow, Exit-LPT temperature, Exit-LPT pressure**

So, let's look at these features closely..

In [ ]:
df.columns

In [ ]:
from seaborn import pairplot
pairplot(df, vars = ['HPSpoolSpeed_RPM', 'LPSpoolSpeed_RPM' , 'FuelFlow_kg/s', 'LPTExitPressureP5_kPA', 'LPTExitTemperatureT5_K', 'Isentr.HPCEfficiency'])

# *Everything is related..*
**Let's have a look at the last row of the Pairplot above..**
Why are these variables correlated like so? Well, even though the engine is an agglomeration of different machines (compressors, turbines, nozzles, etc) everything works together as a hole and components affect each other's opperation.

In this case we are seeing that as HPC efficiency drops, the entire system must compensate for these losses in orther to keep the HPC rotation speed constant. The low pressure compressor rotates faster, increasing the pressure of the air exiting the LPC and entering the HPC. More fuel is burned as the efficiency drops as a way to compensate for the energy loss and this translates into higher temperatures and pressure at the exit of the low pressure turbine.

# A big fat *
Disclaimer: im not an expert, on anything.. so let me know should you notice some error, or if you find better ways to explain the technical details. I will really appretiate that!

**On another subject:**
You may wanna try fitting several regression models to predict HPC efficiency. This is a really easy and simplified example. In reality, all engine components affect each other, all suffer from some level of deterioration, affecting the behaviour of the system. One component may show a decreasse in its performance without being deteriorated, because of the change of working conditions introduced by other deteriorated part of the engine.